In [2]:
!pip install hyperopt

In [3]:
import pandas as pd
import numpy as np
import os
import datetime

import tensorflow as tf
from tensorflow.keras.models import  Sequential
from tensorflow.keras.layers import Conv2D, MaxPool2D, Dense, Flatten, Dropout
from tensorflow.keras.utils import to_categorical

%load_ext tensorboard

import matplotlib.pyplot as plt
from skimage import color, exposure

from sklearn.metrics import accuracy_score

from hyperopt import hp, STATUS_OK, tpe, Trials, fmin

The tensorboard extension is already loaded. To reload it, use:
  %reload_ext tensorboard


In [4]:
cd '/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign'

/content/drive/My Drive/Colab Notebooks/dw_matrix/matrix_three/dw_matrix_road_sign


In [0]:
train = pd.read_pickle('data/train.p')
test = pd.read_pickle('data/test.p')

X_train, y_train = train['features'], train['labels']
X_test, y_test = test['features'], test['labels']

In [0]:
if y_train.ndim == 1: y_train = to_categorical(y_train)
if y_test.ndim == 1: y_test = to_categorical(y_test)

In [0]:
input_shape = X_train.shape[1:]
num_class = y_train.shape[1]

In [0]:
def train_model(model, X_train, y_train, params_fit={}):
  model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])

  logdir = os.path.join("logs", datetime.datetime.now().strftime("%Y%m%d-%H%M%S"))
  tensorboard_callback = tf.keras.callbacks.TensorBoard(logdir, histogram_freq=1)
  
  model.fit(X_train,
            y_train,
            batch_size=params_fit.get('batch_size', 128),
            epochs=params_fit.get('epochs', 5),
            verbose=params_fit.get('verbose', 1),
            validation_data=params_fit.get('validation_data', (X_train, y_train)),
            callbacks= [tensorboard_callback]
  )
  return model

In [0]:
def predict(model_trained, X_test, y_test, scoring=accuracy_score):
  y_test_norm = np.argmax( y_test, axis=1)

  y_pred_prob = model_trained.predict(X_test)
  y_pred = np.argmax( y_pred_prob, axis=1)

  return scoring(y_test_norm, y_pred)

In [0]:
def get_cnn_v5(input_shape, num_class):
  return Sequential([            
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
       MaxPool2D(),
       Dropout(0.3),
       
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
       MaxPool2D(),
       Dropout(0.3),

       Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
       MaxPool2D(),
       Dropout(0.3),

       Flatten(),

       Dense (1024, activation='relu'),
       Dropout(0.3),

       Dense (1024, activation='relu'),
       Dropout(0.3),

       Dense (num_class, activation='softmax')          
])

In [11]:
model = get_cnn_v5(input_shape, num_class)          
model_trained = train_model(model, X_train, y_train)
predict(model_trained, X_test, y_test)

Epoch 1/5
272/272 [==============================] - 9s 34ms/step - loss: 2.3992 - accuracy: 0.3266 - val_loss: 0.8061 - val_accuracy: 0.7609
Epoch 2/5
272/272 [==============================] - 9s 32ms/step - loss: 0.5793 - accuracy: 0.8130 - val_loss: 0.1541 - val_accuracy: 0.9444
Epoch 3/5
272/272 [==============================] - 9s 32ms/step - loss: 0.2284 - accuracy: 0.9317 - val_loss: 0.0509 - val_accuracy: 0.9848
Epoch 4/5
272/272 [==============================] - 9s 32ms/step - loss: 0.1584 - accuracy: 0.9533 - val_loss: 0.0379 - val_accuracy: 0.9895
Epoch 5/5
272/272 [==============================] - 9s 32ms/step - loss: 0.1194 - accuracy: 0.9659 - val_loss: 0.0335 - val_accuracy: 0.9901


0.9682539682539683

In [0]:
def get_model(params):
  return Sequential([            
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', input_shape=input_shape),
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=32, kernel_size=(3, 3), activation='relu', padding='same'),

       MaxPool2D(),
       Dropout(params['dropout_cnn_block_one']),
       
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=64, kernel_size=(3, 3), activation='relu'),
       MaxPool2D(),
       Dropout(params['dropout_cnn_block_two']),

       Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=128, kernel_size=(3, 3), activation='relu', padding='same'),
       Conv2D(filters=128, kernel_size=(3, 3), activation='relu'),
       MaxPool2D(),
       Dropout(params['dropout_cnn_block_three']),

       Flatten(),

       Dense (1024, activation='relu'),
       Dropout(params['dropout_dense_block_one']),

       Dense (1024, activation='relu'),
       Dropout(params['dropout_dense_block_two']),

       Dense (num_class, activation='softmax')          
])

In [0]:
def func_obj(params):
  model = get_model(params)
  model.compile(loss='categorical_crossentropy', optimizer='Adam',metrics=['accuracy'])
 
  model.fit(X_train,
            y_train,
            batch_size=int(params.get('batch_size', 128)),
            epochs=5,
            verbose=0,
            
      )
  score = model.evaluate(X_test, y_test, verbose=0)
  accuracy = score[1]
  print(params, 'accuracy={}'.format(accuracy))

  return {'loss': -accuracy, 'status': STATUS_OK, 'model':model} 

In [14]:
space = {
    'batch size': hp.quniform('batch_size', 100, 200, 10),
    'dropout_cnn_block_one': hp.uniform('dropout_cnn_block_one', 0.3, 0.5),
    'dropout_cnn_block_two': hp.uniform('dropout_cnn_block_two', 0.3, 0.5),
    'dropout_cnn_block_three': hp.uniform('dropout_cnn_block_three', 0.3, 0.5),
    
    'dropout_dense_block_one': hp.uniform('dropout_dense_block_one', 0.3, 0.7),
    'dropout_dense_block_two': hp.uniform('dropout_dense_block_two', 0.3, 0.7),
}

best = fmin(
    func_obj,
    space,
    tpe.suggest,
    30,
    Trials()
)

{'batch size': 200.0, 'dropout_cnn_block_one': 0.48193110763179875, 'dropout_cnn_block_three': 0.3367164668796256, 'dropout_cnn_block_two': 0.36695231279363855, 'dropout_dense_block_one': 0.6193772970581193, 'dropout_dense_block_two': 0.5825531236232272}
accuracy=0.9607709646224976
{'batch size': 170.0, 'dropout_cnn_block_one': 0.47224279678219416, 'dropout_cnn_block_three': 0.493920455920829, 'dropout_cnn_block_two': 0.4808505549307221, 'dropout_dense_block_one': 0.5768283770240008, 'dropout_dense_block_two': 0.31650512449725227}
accuracy=0.9113378524780273
{'batch size': 180.0, 'dropout_cnn_block_one': 0.4145289245659869, 'dropout_cnn_block_three': 0.4024781775153688, 'dropout_cnn_block_two': 0.44237650123557826, 'dropout_dense_block_one': 0.42031135396982955, 'dropout_dense_block_two': 0.40108144428630577}
accuracy=0.9625850319862366
{'batch size': 130.0, 'dropout_cnn_block_one': 0.45764557446146104, 'dropout_cnn_block_three': 0.4534530505035177, 'dropout_cnn_block_two': 0.435697859